Ejercicio: sesgo del modelo de datos desequilibrados

En este ejercicio, analizaremos más de cerca los conjuntos de datos desequilibrados, qué efectos tienen en las predicciones y cómo se pueden abordar.

También emplearemos matrices de confusión para evaluar las actualizaciones del modelo.

Visualización de datos

Al igual que en el ejercicio anterior, usamos un conjunto de datos que representa diferentes clases de objetos que se encuentran en la montaña:

In [ ]:
import pandas
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/snow_objects.csv
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/snow_objects_balanced.csv

#Import the data from the .csv file
dataset = pandas.read_csv('snow_objects.csv', delimiter="\t")

# Let's have a look at the data
dataset

Recuerde que tenemos un *conjunto de datos desequilibrado*. Algunas clases son mucho más frecuentes que otras:

In [ ]:
import graphing # custom graphing code. See our GitHub repo for details

# Plot a histogram with counts for each label
graphing.multiple_histogram(dataset, label_x="label", label_group="label", title="Label distribution")

Usando la clasificación binaria

Para este ejercicio construiremos un modelo de clasificación binaria. Queremos predecir si los objetos en la nieve son "excursionistas" o "no excursionistas".

Para hacer eso, primero debemos agregar otra columna a nuestro conjunto de datos y establecerla en True donde la etiqueta original es excursionista, y False en cualquier otra cosa:

In [ ]:
# Add a new label with true/false values to our dataset
dataset["is_hiker"] = dataset.label == "hiker"

# Plot frequency for new label
graphing.multiple_histogram(dataset, label_x="is_hiker", label_group="is_hiker", title="Distribution for new binary label 'is_hiker'")

Ahora tenemos solo dos clases de etiquetas en nuestro conjunto de datos, pero lo hemos hecho aún más desequilibrado.

Entrenemos el modelo de bosque aleatorio usando is_hiker como la variable de destino, luego midamos su precisión en los conjuntos de entrenamiento y prueba:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
# import matplotlib.pyplot as plt
# from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import accuracy_score

# Custom function that measures accuracy on different models and datasets
# We will use this in different parts of the exercise
def assess_accuracy(model, dataset, label):
    """
    Asesses model accuracy on different sets
    """ 
    actual = dataset[label]        
    predictions = model.predict(dataset[features])
    acc = accuracy_score(actual, predictions)
    return acc

# Split the dataset in an 70/30 train/test ratio. 
train, test = train_test_split(dataset, test_size=0.3, random_state=1, shuffle=True)

# define a random forest model
model = RandomForestClassifier(n_estimators=1, random_state=1, verbose=False)

# Define which features are to be used (leave color out for now)
features = ["size", "roughness", "motion"]

# Train the model using the binary label
model.fit(train[features], train.is_hiker)

print("Train accuracy:", assess_accuracy(model,train, "is_hiker"))
print("Test accuracy:", assess_accuracy(model,test, "is_hiker"))

La precisión se ve bien tanto para el entrenamiento como para los conjuntos de prueba, pero recuerde que esta métrica no es una medida absoluta del éxito.

Deberíamos trazar una matriz de confusión para ver cómo funciona realmente el modelo:

In [ ]:
# sklearn has a very convenient utility to build confusion matrices
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

# Calculate the model's accuracy on the TEST set
actual = test.is_hiker
predictions = model.predict(test[features])

# Build and print our confusion matrix, using the actual values and predictions 
# from the test set, calculated in previous cells
cm = confusion_matrix(actual, predictions, normalize=None)

# Create the list of unique labels in the test set, to use in our plot
# I.e., ['True', 'False',]
unique_targets = sorted(list(test["is_hiker"].unique()))

# Convert values to lower case so the plot code can count the outcomes
x = y = [str(s).lower() for s in unique_targets]

# Plot the matrix above as a heatmap with annotations (values) in its cells
fig = ff.create_annotated_heatmap(cm, x, y)

# Set titles and ordering
fig.update_layout(  title_text="<b>Confusion matrix</b>", 
                    yaxis = dict(categoryorder = "category descending")
                    )

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted label",
                        xref="paper",
                        yref="paper"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.15,
                        y=0.5,
                        showarrow=False,
                        text="Actual label",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# We need margins so the titles fit
fig.update_layout(margin=dict(t=80, r=20, l=120, b=50))
fig['data'][0]['showscale'] = True
fig.show()


La matriz de confusión nos muestra que, a pesar de las métricas reportadas, el modelo no es increíblemente preciso.

De las 660 muestras presentes en el conjunto de prueba (30% del total de muestras), predijo 29 falsos negativos y 33 falsos positivos.

Más importante aún, mire la fila inferior, que muestra lo que sucedió cuando al modelo se le mostró información sobre un excursionista: se equivocó en la respuesta casi el 30% de las veces. ¡Esto significa que no identificaría correctamente a casi el 30% de las personas en la montaña!

¿Qué sucede si usamos este modelo para hacer predicciones sobre conjuntos balanceados?

Carguemos un conjunto de datos con la misma cantidad de resultados para "excursionistas" y "no excursionistas", luego usemos esos datos para hacer predicciones:

In [ ]:
# Load and print umbiased set
#Import the data from the .csv file
balanced_dataset = pandas.read_csv('snow_objects_balanced.csv', delimiter="\t")

#Let's have a look at the data
graphing.multiple_histogram(balanced_dataset, label_x="label", label_group="label", title="Label distribution")

Este nuevo conjunto de datos está equilibrado entre las clases, pero para nuestros propósitos queremos que esté equilibrado entre excursionistas y no excursionistas.

Para simplificar, tomemos a los excursionistas más una muestra aleatoria de los no excursionistas.

In [ ]:
# Add a new label with true/false values to our dataset
balanced_dataset["is_hiker"] = balanced_dataset.label == "hiker"

hikers_dataset = balanced_dataset[balanced_dataset["is_hiker"] == 1] 
nonhikers_dataset = balanced_dataset[balanced_dataset["is_hiker"] == False] 
# take a random sampling of non-hikers the same size as the hikers subset
nonhikers_dataset = nonhikers_dataset.sample(n=len(hikers_dataset.index), random_state=1)
balanced_dataset = pandas.concat([hikers_dataset, nonhikers_dataset])

# Plot frequency for "is_hiker" labels
graphing.multiple_histogram(balanced_dataset, label_x="is_hiker", label_group="is_hiker", title="Label distribution in balanced dataset")

Como puede ver, la etiqueta is_hiker tiene el mismo número de Verdadero y Falso para ambas clases. Ahora estamos usando un conjunto de datos balanceado de clases.

Ejecutemos predicciones en este conjunto usando el modelo previamente entrenado:

In [ ]:
# Test the model using a balanced dataset
actual = balanced_dataset.is_hiker
predictions = model.predict(balanced_dataset[features])

# Build and print our confusion matrix, using the actual values and predictions 
# from the test set, calculated in previous cells
cm = confusion_matrix(actual, predictions, normalize=None)

# Print accuracy using this set
print("Balanced set accuracy:", assess_accuracy(model,balanced_dataset, "is_hiker"))

Como era de esperar, vemos una caída notable en la precisión al usar un conjunto diferente.

Nuevamente, analicemos visualmente su desempeño:

In [ ]:
# plot new confusion matrix
# Create the list of unique labels in the test set to use in our plot
unique_targets = sorted(list(balanced_dataset["is_hiker"].unique()))

# Convert values to lower case so the plot code can count the outcomes
x = y = [str(s).lower() for s in unique_targets]

# Plot the matrix above as a heatmap with annotations (values) in its cells
fig = ff.create_annotated_heatmap(cm, x, y)

# Set titles and ordering
fig.update_layout(  title_text="<b>Confusion matrix</b>", 
                    yaxis = dict(categoryorder = "category descending")
                    )

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted label",
                        xref="paper",
                        yref="paper"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.15,
                        y=0.5,
                        showarrow=False,
                        text="Actual label",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# We need margins so the titles fit
fig.update_layout(margin=dict(t=80, r=20, l=120, b=50))
fig['data'][0]['showscale'] = True
fig.show()

La matriz de confusión confirma la poca precisión con este conjunto de datos, pero ¿por qué sucede esto cuando teníamos métricas tan excelentes en los conjuntos de pruebas y trenes anteriores?

Recuérdese que el primer modelo estaba muy desequilibrado. La clase "excursionista" representó aproximadamente el 22% de los resultados.

Cuando ocurre tal desequilibrio, los modelos de clasificación no tienen suficientes datos para aprender los patrones de la clase minoritaria y, como consecuencia, se sesgan hacia la clase mayoritaria.

Los conjuntos desequilibrados se pueden abordar de varias maneras:

Mejora de la selección de datos<br>
Remuestreo del conjunto de datos<br>
Uso de clases ponderadas<br>
Para este ejercicio, nos centraremos en la última opción.

Uso de ponderaciones de clase para equilibrar el conjunto de datos
Podemos asignar diferentes pesos a las clases mayoritarias y minoritarias, según su distribución, y modificar nuestro algoritmo de entrenamiento para que tenga en cuenta esa información durante la fase de entrenamiento.

Luego, penalizará los errores cuando la clase minoritaria esté mal clasificada, en esencia, "forzando" al modelo a aprender mejor sus características y patrones.

Para usar clases ponderadas, tenemos que volver a entrenar nuestro modelo usando el conjunto de trenes original, pero esta vez diciéndole al algoritmo que use pesos al calcular los errores:

In [ ]:
# Import function used in calculating weights
from sklearn.utils import class_weight

# Retrain model using class weights
# Using class_weight="balanced" tells the algorithm to automatically calculate weights for us
weighted_model = RandomForestClassifier(n_estimators=1, random_state=1, verbose=False, class_weight="balanced")
# Train the weighted_model using binary label
weighted_model.fit(train[features], train.is_hiker)

print("Train accuracy:", assess_accuracy(weighted_model,train, "is_hiker"))
print("Test accuracy:", assess_accuracy(weighted_model, test, "is_hiker"))

Después de usar las clases ponderadas, la precisión del tren se mantuvo casi igual, mientras que la precisión de la prueba mostró una pequeña mejora (aproximadamente el 1 %).

Veamos si los resultados mejoran usando el conjunto balanceado para predicciones nuevamente:

In [ ]:
print("Balanced set accuracy:", assess_accuracy(weighted_model, balanced_dataset, "is_hiker"))

# Test the weighted_model using a balanced dataset
actual = balanced_dataset.is_hiker
predictions = weighted_model.predict(balanced_dataset[features])

# Build and print our confusion matrix, using the actual values and predictions 
# from the test set, calculated in previous cells
cm = confusion_matrix(actual, predictions, normalize=None)


La precisión del conjunto equilibrado aumentó aproximadamente un 4 %, pero aun así deberíamos tratar de visualizar y comprender los nuevos resultados.

Matriz de confusión final
Ahora podemos trazar una matriz de confusión final, que representa predicciones para un conjunto de datos equilibrado, utilizando un modelo entrenado en un conjunto de datos de clase ponderada:

In [ ]:
# Plot the matrix above as a heatmap with annotations (values) in its cells
fig = ff.create_annotated_heatmap(cm, x, y)

# Set titles and ordering
fig.update_layout(  title_text="<b>Confusion matrix</b>", 
                    yaxis = dict(categoryorder = "category descending")
                    )

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=0.5,
                        y=-0.15,
                        showarrow=False,
                        text="Predicted label",
                        xref="paper",
                        yref="paper"))

fig.add_annotation(dict(font=dict(color="black",size=14),
                        x=-0.15,
                        y=0.5,
                        showarrow=False,
                        text="Actual label",
                        textangle=-90,
                        xref="paper",
                        yref="paper"))

# We need margins so the titles fit
fig.update_layout(margin=dict(t=80, r=20, l=120, b=50))
fig['data'][0]['showscale'] = True
fig.show()

Si bien los resultados pueden parecer un poco decepcionantes, ahora tenemos un 21 % de predicciones incorrectas (FN + FP), frente al 25 % del experimento anterior.

Las predicciones correctas (TPs + TNs) pasaron del 74,7% al 78,7%.

¿Es significativa o no una mejora general del 4%?

Recuerde que teníamos relativamente pocos datos para entrenar el modelo, y las funciones que tenemos disponibles aún pueden ser tan similares para diferentes muestras (por ejemplo, los excursionistas y los animales tienden a ser pequeños, no bruscos y se mueven mucho), que a pesar de nuestra esfuerzos, el modelo todavía tiene algunas dificultades para hacer predicciones correctas.

Solo tuvimos que cambiar una sola línea de código para obtener mejores resultados, ¡así que parece que vale la pena el esfuerzo!

Resumen

Este fue un ejercicio largo, donde cubrimos los siguientes temas:

Creando nuevos campos de etiqueta para que podamos realizar una clasificación binaria usando un conjunto de datos con múltiples clases.<br>
Cómo el entrenamiento en conjuntos desequilibrados puede tener un efecto negativo en el rendimiento, especialmente cuando se utilizan datos ocultos de conjuntos de datos equilibrados.<br>
Evaluación de resultados de modelos de clasificación binaria utilizando una matriz de confusión.<br>
Uso de clases ponderadas para abordar los desequilibrios de clase al entrenar un modelo y evaluar los resultados.